In [13]:
import pandas as pd

# Install scikit-learn
!pip install scikit-learn
# Install openpyxl
!pip install openpyxl


from sklearn.model_selection import (
    train_test_split,
)  # used to split your dataset into two parts: a training set and a testing set.
from sklearn.linear_model import (
    LinearRegression,
)  # used to create a linear regression model.
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)  # evaluate the performance of machine learning models
import numpy as np



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Load dataset
df = pd.read_excel('Gold_data_filtered.xlsx')

In [3]:
# Display the first few rows
print(df.head())

# Check for any missing values
print(df.isnull().sum())

        date         open         high          low        close  volume
0 2019-03-01  1312.199951  1312.199951  1288.900024  1296.400024     110
1 2019-03-04  1285.500000  1287.000000  1281.900024  1284.800049      28
2 2019-03-05  1287.000000  1287.000000  1282.000000  1282.000000     114
3 2019-03-06  1287.099976  1287.099976  1282.599976  1284.900024      14
4 2019-03-07  1281.000000  1284.599976  1281.000000  1283.800049      11
date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64


In [4]:
# Define the features (X) and the target (y)
X = df[['open', 'high', 'low', 'volume']]
y = df['close']

In [6]:
# Function to split dataset into train, test, and validation sets
def split_data(X, y, train_ratio, test_ratio, val_ratio):
    # First split: train + (test + validation)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(1 - train_ratio), random_state=42)
    
    # Calculate the ratio for test within the remaining (test + validation) split
    test_size = test_ratio / (test_ratio + val_ratio)
    
    # Second split: test and validation from the temporary set
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=(1 - test_size), random_state=42)
    
    return X_train, X_test, X_val, y_train, y_test, y_val

In [12]:
# List of ratio configurations
ratios = {
    "7:2:1": (0.7, 0.2, 0.1),
    "6:3:1": (0.6, 0.3, 0.1),
    "5:3:2": (0.5, 0.3, 0.2)
}

In [14]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, mape, rmse


In [15]:
# Loop through each ratio, build the model, and evaluate
for ratio_name, (train_ratio, test_ratio, val_ratio) in ratios.items():
    print(f"\nEvaluating ratio {ratio_name}")

    # Split the data
    X_train, X_test, X_val, y_train, y_test, y_val = split_data(
        X, y, train_ratio, test_ratio, val_ratio
    )

    # Initialize and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Validation evaluation
    y_val_pred = model.predict(X_val)
    val_mae, val_mape, val_rmse = evaluate_model(y_val, y_val_pred)

    print(
        f"Validation Set - MAE: {val_mae:.4f}, MAPE: {val_mape:.4f}, RMSE: {val_rmse:.4f}"
    )

    # Test evaluation
    y_test_pred = model.predict(X_test)
    test_mae, test_mape, test_rmse = evaluate_model(y_test, y_test_pred)

    print(
        f"Test Set - MAE: {test_mae:.4f}, MAPE: {test_mape:.4f}, RMSE: {test_rmse:.4f}"
    )


Evaluating ratio 7:2:1
Validation Set - MAE: 3.9294, MAPE: 0.0021, RMSE: 5.7310
Test Set - MAE: 4.5479, MAPE: 0.0025, RMSE: 6.7112

Evaluating ratio 6:3:1
Validation Set - MAE: 4.6897, MAPE: 0.0025, RMSE: 6.9391
Test Set - MAE: 3.9912, MAPE: 0.0022, RMSE: 5.7895

Evaluating ratio 5:3:2
Validation Set - MAE: 4.0592, MAPE: 0.0022, RMSE: 5.8925
Test Set - MAE: 4.3230, MAPE: 0.0024, RMSE: 6.2827
